In [1]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import os, pickle
os.chdir('/Users/kathleenhamilton/Desktop/QHACK2021/qose/QHACK2021/src')
from subarchitecture_tree_search import *

## First example

The first example builds a circuit classifier for the moons dataset available in sklearn.  The architecture consists of:

`Encoding` -> `Parameterized Layers` -> `One Hot Encoding`

The tree will iteratively grow the parameterized layers through the addition of:
* Layers of RX gates
* Layers of ZZ gates
* Layers of RY gates

In [3]:
# Create a unique name for your experiment
EXPERIMENT_NAME = 'one_hot'

# Create a directory to store the data
if not os.path.exists('data'):
    os.mkdir('data/')

data_path = f'data/{EXPERIMENT_NAME}'
if not os.path.exists(data_path):
    os.mkdir(data_path)

# Create a configuration file for the tree prune algorithm
config = {'nqubits': 4,
          'min_tree_depth': 4,
          'max_tree_depth': 8,
          'prune_rate': 0.15,
          'prune_step': 3,
          'plot_trees': False,
          'data_set': 'moons',
          'nsteps': 5,
          'opt': qml.AdamOptimizer,
          'batch_size': [8,16,32,64],
          'n_samples': 1500,
          'learning_rate': [0.001,0.005,0.01],
          'save_frequency': 1,
          'save_path': data_path,
          'save_timing': True
          }

# Save the configuration file so that we can remember what we did
with open(data_path + '/config.pickle', 'wb') as f:
    pickle.dump(config, f)

In [ ]:
# Execute the algorithm
run_tree_architecture_search(config)

saving timing info
Depth = 1
current graph:  [('ROOT', {'W': 0.0}), ('E1', {'W': 1.0})]
Depth = 2
Current best architecture:  E1
max W: 1.0
Depth = 3
Current best architecture:  E1:ZZ
max W: 1.7065785107969043
Depth = 4
Prune Tree
Current best architecture:  E1:ZZ:X
max W: 2.0478665579541575
Grow Pruned Tree
Depth = 5
Grow Tree
Current best architecture:  E1:ZZ:X
max W: 2.0478665579541575
Depth = 6
Grow Tree
Current best architecture:  E1:ZZ:Y:X:Y
max W: 2.559757666244391


### Try running a loop over some hyper_parameters

This uses the circuit classifer built during the Challenge -- the circuit and QNode is constructed inside the function `classify_data`

The following characteristics are hard wired inside the function `classify_data`:
* number of qubits (3)
* number of rotation gates (6)
* the initialization used for the rotations and weights (rotations intialized with 0, weights initialized with random numbers drawn uniformly from $[-2.5,2.5]$
* the rotation gates that are trained (RY)
* the rotation gates used in the `AngleEmbeddding` (RY)
* the optimizer (`AdamOptimizer`)

The following parameters are passed as keywords:
* `s` (the number of steps to train for)
* `batch_size` (the batch size used in training)
* `learning_rate` (the initial learning rate for Adam)

As in (de Wynter 2020) we only train each circuit for a few steps (here I'm using 10).  In (de Wynter 2020) the error rate surrogate is defined using a loss function evaluated over a subset of the data -- here I'm using the accuracy of assigned labels over the test data.  The full number of samples that I generated for the dataset is given by `n_samples` (above).  I've split that data in to train,test sets. 

### Try running a loop over some hyper_parameters

This time use a circuit (QNode) that is created outside the function and passed as an argument

Things that are still hard-wired inside the `train_circuit` function:
* Optimizer choice
* Initialization choice (same as above)


In [ ]:
def variational_circuit(params,features=None):
    r_ = QUBITS//len(features)
    large_features = np.tile(features,r_)
    qml.templates.embeddings.AngleEmbedding(large_features, wires=range(QUBITS), rotation='Y') # replace with more general embedding
    if len(params)%QUBITS!=0:
        print('chooose parameter length that is divisible by number of qubits')
        return
    W= np.reshape(params,(len(params)//QUBITS,QUBITS))
    qml.templates.layers.BasicEntanglerLayers(W, wires=range(QUBITS), rotation=qml.ops.RY)
    return [qml.expval(qml.PauliZ(idx)) for idx in range(QUBITS)]


In [ ]:
qubit_sizes=[2,4]
batch_sets = [2,4,8,16,32]
learning_rates = [0.001,0.005,0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
results = {}
Wmax = 0.0
for qdx in qubit_sizes:
    QUBITS=qdx
    dev = qml.device("default.qubit",wires=QUBITS)
    # Instantiate the QNode
    outside_circuit = qml.QNode(func=variational_circuit,device=dev)
    n_params=2*QUBITS
    for idx,sdx in hyperparameters:
        p,i,er,wtemp,weights=train_circuit(outside_circuit,n_params,qdx,X_train,Y_train,s=5,batch_size=idx,rate_type='accuracy',learning_rate=sdx)
        print(p,i,er,wtemp,weights)
        if wtemp>=Wmax:
            Wmax=wtemp
            saved_weights = weights
            output = (idx,sdx,p,i,er)
print("Max W coef: ", Wmax)
print("saved weights: ",saved_weights)
print("hyperparameters: ",output[0],output[1])
print("variables (p,i,er): ",p,i,er)

In [ ]:
loop_over_hyperparameters(outside_circuit,4,X_train,Y_train,batch_sets=[2,4,8,16,32],learning_rates=[0.001,0.005,0.01,0.05],s=5,rate_type='accuracy')

### Pull in more detailed circuit design

In [ ]:
import sys
import pennylane as qml
import sklearn as skl
import autograd.numpy as np
import itertools
import time

In [ ]:


def zz_layer(wires, params):
    nq = len(wires)
    for n in range(nq - 1):
        zz_gate([n, n + 1], params[n])
    zz_gate([nq - 1, 0], params[nq - 1])


def zz_gate(wires, gamma):
    if isinstance(gamma,(float,qml.variable.Variable)):
        qml.CNOT(wires=wires)
        qml.RZ(gamma, wires=wires[1])
        qml.CNOT(wires=wires)
    else:
        qml.CNOT(wires=wires)
        qml.RZ(gamma._value, wires=wires[1])
        qml.CNOT(wires=wires)

def CNOT_layer(wires):
    nq = len(wires)
    for n in range(nq-1):
        qml.CNOT(wires=[n,n+1])
    qml.CNOT(wires=[nq-1,0])

def x_layer(wires, params):
    nqubits = len(wires)
    if isinstance(params,(list,np.ndarray,qml.variable.Variable)):
        for n in range(nqubits):
            qml.RX(params[n], wires=[n, ])
    else:
        for n in range(nqubits):
            qml.RX(params[n]._value, wires=[n, ])


def y_layer(wires, params):
    nqubits = len(wires)
    if isinstance(params,(list,np.ndarray,qml.variable.Variable)):
        for n in range(nqubits):
            qml.RY(params[n], wires=[n, ])
    else:
        for n in range(nqubits):
            qml.RY(params[n]._value, wires=[n, ])

#TODO: ADD W-COST HERE

string_to_layer_mapping = {'ZZ': zz_layer, 'X': x_layer, 'Y': y_layer}

In [ ]:

def string2circuit(gate_params=[],features=[],arch_params=[],n_wires=None):
    string_to_param_count = {'E1':0,'ZZ':n_wires,'X':n_wires,'Y':n_wires}
    pdx = 0
    for gdx in arch_params:
        if gdx=='E1':
            r_ = n_wires//len(features)
            large_features = np.tile(features,r_)
            qml.templates.embeddings.AngleEmbedding(large_features, wires=range(n_wires), rotation='Y') # replace with more general embedding
            qml.templates.embeddings.AngleEmbedding(features, wires=range(n_wires), rotation='Y') # replace with more general embedding
        elif gdx=='ZZ':
            #w = gate_params[pdx:pdx+n_wires]
            #zz_layer(range(n_wires),w)
            CNOT_layer(range(n_wires))
            #pdx+=n_wires
        elif gdx=='X':
            w = gate_params[pdx:pdx+n_wires]
            x_layer(range(n_wires),w)
            pdx+=n_wires
        elif gdx=='Y':
            w = gate_params[pdx:pdx+n_wires]
            y_layer(range(n_wires),w)
            pdx+=n_wires
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_wires)]
    

In [ ]:
batch_sets = [2,4]
learning_rates = [0.001,0.005]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
results = {}
Wmax = 0.0

ARCH = ['E1','ZZ','Y','ZZ','Y','ZZ','Y','ZZ','Y','ZZ','Y']
WIRES = 2
def temp_circuit(w,features=None): return string2circuit(gate_params=w,features=features,arch_params=ARCH,n_wires=WIRES)

 
dev = qml.device("default.qubit",wires=WIRES)
outside_circuit = qml.QNode(temp_circuit,device=dev)

#if using zz-layer, use this line
#n_params=np.sum([WIRES for x in ARCH if (x!='E1')])
#if useing CNOT-layer, use this line
n_params=np.sum([WIRES for x in ARCH if (x=='X') or (x=='Y')])
print(n_params)
for idx,sdx in hyperparameters:
    p,i,er,wtemp,weights=train_circuit(outside_circuit,n_params,WIRES,X_train,Y_train,s=5,batch_size=idx,rate_type='batch_cost',learning_rate=sdx)
    print('final weights:',weights)
    print(p,i,er,wtemp,weights)
    if wtemp>=Wmax:
        Wmax=wtemp
        saved_weights = weights
        output = (idx,sdx,p,i,er)
print("Max W coef: ", Wmax)
print("saved weights: ",saved_weights)
print("hyperparameters: ",output[0],output[1])
print("variables (p,i,er): ",p,i,er)